## **Análisis de Calidad del Aire**

1. Carga los datos demográficos en una tabla utilizando pandas, utilizando el mismo CSV de demografía del ejemplo estudiado en clases.

In [2]:
# Importes
import pandas as pd
import requests
from typing import Set

In [ ]:
# Cargar los datos demográficos
url = "https://public.opendatasoft.com/explore/dataset/us-cities-demographics/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B"
demografia_df = pd.read_csv(url, sep=';')

# Eliminar columnas no deseadas
demografia_df = demografia_df.drop(columns=['Race', 'Count', 'Number of Veterans'])

# Eliminar filas duplicadas
demografia_df = demografia_df.drop_duplicates()

demografia_df.to_csv('demografia_df.csv', index=False)

In [3]:
# Leer el .csv
pd.read_csv('demografia_df.csv').head(5)

,City,State,Median Age,Male Population,Female Population,Total Population,Foreign-born,Average Household Size,State Code
0,Framingham,Massachusetts,40.3,35442.0,35768.0,71210,19070.0,2.49,MA
1,Waco,Texas,29.3,63452.0,68890.0,132342,14235.0,2.57,TX
2,Fort Worth,Texas,32.6,414126.0,422843.0,836969,143404.0,2.88,TX
3,Bellingham,Washington,30.7,41286.0,43857.0,85143,8713.0,2.44,WA
4,Maple Grove,Minnesota,38.6,31780.0,36601.0,68381,7645.0,2.64,MN


2. Parsear los datos de calidad del aire para cada ciudad en la tabla demográfica obteniendo la información con la API https://api-ninjas.com/api/airquality. Crear una tabla de dimensiones utilizando pandas para almacenar estos datos.

- Tomar el elemento **concentration** de cada entrada por fila.

In [4]:
# Configurar la API y los encabezados
api_url = "https://api.api-ninjas.com/v1/airquality"
api_key = "DFOVFVUvKoQjatRGySL2GQ==ARhIOtQCKOuju0of"
headers = {'X-Api-Key': api_key}
calidad_aire_data = []

for city in pd.read_csv('demografia_df.csv')['City']:
    response = requests.get(api_url, headers=headers, params={'city': city})
    if response.status_code == 200:
        data = response.json()
        concentrations = {key: value.get('concentration') for key, value in data.items() if key != 'overall_aqi'}  # Obtener las concentraciones de cada parámetro
        concentrations['City'] = city  # Agregar el nombre de la ciudad al diccionario de concentraciones
        calidad_aire_data.append(concentrations)
    else:
        print(f"Error al conseguir datos de: {city}, Código de error: {response.status_code}")

# Crear un DataFrame con los datos de calidad del aire
calidad_aire_df = pd.DataFrame(calidad_aire_data)

# Reordenar las columnas para que la primera sea 'City'
calidad_aire_df = calidad_aire_df[['City', 'CO', 'PM10', 'SO2', 'PM2.5', 'O3', 'NO2']]

calidad_aire_df.to_csv('calidad_aire_df.csv', index=False)

pd.read_csv('calidad_aire_df.csv').head(5)

Error al conseguir datos de: Urban Honolulu, Código de error: 400
Error al conseguir datos de: Augusta-Richmond County consolidated government, Código de error: 400
Error al conseguir datos de: Florence-Graham, Código de error: 400
Error al conseguir datos de: Louisville/Jefferson County metro government, Código de error: 400


,City,CO,PM10,SO2,PM2.5,O3,NO2
0,Framingham,320.44,7.61,3.34,5.98,41.13,27.76
1,Waco,186.92,2.67,0.29,1.75,76.53,2.27
2,Fort Worth,226.97,5.54,2.44,4.54,113.01,6.34
3,Bellingham,250.34,13.75,3.07,12.91,97.28,10.11
4,Maple Grove,297.07,4.52,0.60,4.25,46.49,11.65
